# <span style="color:#00bfff">Proyecto: Clasificación de Emociones y Sentimientos</span>

## <span style="color:#00bfff">💡 Problema a Resolver</span>

Predecir la **emoción predominante** en una oración dada, a partir del dataset `combined_emotion.csv`, usando técnicas de clasificación supervisada.

También incluiremos clasificación de sentimiento (positivo/negativo) con `combined_sentiment_data.csv`.

---

### <span style="color:#00bfff">Objetivos:</span>
1. **Análisis Exploratorio de Datos (EDA)**
2. **Limpieza y Preprocesamiento de Datos**
3. **Preprocesamiento de Texto**
4. **Entrenamiento de Modelos de Clasificación**
5. **Evaluación y Comparación de Modelos**

## <span style="color:#00bfff">1. Importación de Librerías</span>

In [2]:
# Cargar variables de entorno
from dotenv import load_dotenv
import os
from pathlib import Path

# Cargar .env
load_dotenv()

# Configuración desde .env
DATA_EMOTION_PATH = os.getenv('DATA_EMOTION_PATH', 'dataEmotion/combined_emotion.csv')
DATA_SENTIMENT_PATH = os.getenv('DATA_SENTIMENT_PATH', 'dataEmotion/combined_sentiment_data.csv')

MAX_FEATURES = int(os.getenv('MAX_FEATURES', 5000))
MIN_DF = int(os.getenv('MIN_DF', 2))
MAX_DF = float(os.getenv('MAX_DF', 0.9))
NGRAM_RANGE = (int(os.getenv('NGRAM_RANGE_MIN', 1)), int(os.getenv('NGRAM_RANGE_MAX', 2)))

TEST_SIZE = float(os.getenv('TEST_SIZE', 0.2))
RANDOM_STATE = int(os.getenv('RANDOM_STATE', 42))

KNN_NEIGHBORS = int(os.getenv('KNN_NEIGHBORS', 5))
RF_ESTIMATORS = int(os.getenv('RF_ESTIMATORS', 100))
LR_MAX_ITER = int(os.getenv('LR_MAX_ITER', 1000))

# Crear carpeta de modelos si no existe
MODELS_DIR = Path('models')
MODELS_DIR.mkdir(exist_ok=True)

print("✓ Configuración cargada desde .env")
print(f"  - Random State: {RANDOM_STATE}")
print(f"  - Test Size: {TEST_SIZE}")
print(f"  - Max Features: {MAX_FEATURES}")
print(f"  - Carpeta de modelos: {MODELS_DIR}")

✓ Configuración cargada desde .env
  - Random State: 42
  - Test Size: 0.2
  - Max Features: 5000
  - Carpeta de modelos: models


## <span style="color:#00bfff">0. Configuración del Proyecto</span>

Cargamos las variables de entorno desde el archivo `.env`

In [1]:
# Librerías básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesamiento
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

# Procesamiento de texto
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Modelos de clasificación
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Evaluación
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
import warnings
warnings.filterwarnings('ignore')

ImportError: cannot import name '_is_pandas_dataframe' from 'matplotlib.cbook' (c:\Users\julia\anaconda3\envs\unidad3\Lib\site-packages\matplotlib\cbook\__init__.py)

## <span style="color:#00bfff">2. Carga de Datos</span>

In [ ]:
# Cargar datasets usando variables de configuración
df_emotion = pd.read_csv(DATA_EMOTION_PATH)
df_sentiment = pd.read_csv(DATA_SENTIMENT_PATH)

print("="*60)
print("DATASET DE EMOCIONES")
print("="*60)
print(f"Dimensiones: {df_emotion.shape}")
print(f"\nPrimeras filas:")
display(df_emotion.head())

print("\n" + "="*60)
print("DATASET DE SENTIMIENTOS")
print("="*60)
print(f"Dimensiones: {df_sentiment.shape}")
print(f"\nPrimeras filas:")
display(df_sentiment.head())

## <span style="color:#00bfff">3. Análisis Exploratorio de Datos (EDA)</span>

### <span style="color:#00bfff">3.1 Información General del Dataset de Emociones</span>

In [ ]:
# Información del dataset
print("Información del Dataset de Emociones:")
print(df_emotion.info())
print("\n" + "="*60)

# Verificar valores nulos
print("\nValores nulos por columna:")
print(df_emotion.isnull().sum())
print("\n" + "="*60)

# Estadísticas descriptivas
print("\nEstadísticas descriptivas:")
print(df_emotion.describe())

In [ ]:
# Distribución de emociones
print("Distribución de Emociones:")
print(df_emotion['emotion'].value_counts())
print("\n" + "="*60)

# Visualización de la distribución
plt.figure(figsize=(10, 6))
df_emotion['emotion'].value_counts().plot(kind='bar', color='skyblue')
plt.title('Distribución de Emociones en el Dataset', fontsize=14, fontweight='bold')
plt.xlabel('Emoción', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

### <span style="color:#00bfff">3.2 Análisis de Longitud de Texto</span>

In [ ]:
# Agregar columna con longitud de las oraciones
df_emotion['sentence_length'] = df_emotion['sentence'].apply(lambda x: len(str(x).split()))

# Estadísticas de longitud
print("Estadísticas de Longitud de Oraciones:")
print(df_emotion['sentence_length'].describe())

# Visualización
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(df_emotion['sentence_length'], bins=50, color='coral', alpha=0.7, edgecolor='black')
plt.title('Distribución de Longitud de Oraciones', fontsize=12, fontweight='bold')
plt.xlabel('Número de Palabras', fontsize=10)
plt.ylabel('Frecuencia', fontsize=10)
plt.grid(axis='y', alpha=0.3)

plt.subplot(1, 2, 2)
df_emotion.groupby('emotion')['sentence_length'].mean().sort_values().plot(kind='barh', color='teal')
plt.title('Longitud Media por Emoción', fontsize=12, fontweight='bold')
plt.xlabel('Longitud Media (palabras)', fontsize=10)
plt.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

### <span style="color:#00bfff">3.3 Análisis del Dataset de Sentimientos</span>

In [ ]:
# Información del dataset de sentimientos
print("Información del Dataset de Sentimientos:")
print(df_sentiment.info())
print("\n" + "="*60)

# Verificar valores nulos
print("\nValores nulos por columna:")
print(df_sentiment.isnull().sum())
print("\n" + "="*60)

# Distribución de sentimientos
print("\nDistribución de Sentimientos:")
print(df_sentiment['sentiment'].value_counts())

# Visualización
plt.figure(figsize=(8, 5))
df_sentiment['sentiment'].value_counts().plot(kind='bar', color=['green', 'red'])
plt.title('Distribución de Sentimientos en el Dataset', fontsize=14, fontweight='bold')
plt.xlabel('Sentimiento', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## <span style="color:#00bfff">4. Limpieza y Preprocesamiento de Datos</span>

### <span style="color:#00bfff">4.1 Tratamiento de Valores Nulos</span>

In [ ]:
# Verificar y eliminar valores nulos en el dataset de emociones
print(f"Valores nulos antes de limpieza (Emociones): {df_emotion.isnull().sum().sum()}")
df_emotion_clean = df_emotion.dropna()
print(f"Valores nulos después de limpieza (Emociones): {df_emotion_clean.isnull().sum().sum()}")
print(f"Filas eliminadas: {len(df_emotion) - len(df_emotion_clean)}")
print("\n" + "="*60)

# Verificar y eliminar valores nulos en el dataset de sentimientos
print(f"\nValores nulos antes de limpieza (Sentimientos): {df_sentiment.isnull().sum().sum()}")
df_sentiment_clean = df_sentiment.dropna()
print(f"Valores nulos después de limpieza (Sentimientos): {df_sentiment_clean.isnull().sum().sum()}")
print(f"Filas eliminadas: {len(df_sentiment) - len(df_sentiment_clean)}")

### <span style="color:#00bfff">4.2 Limpieza de Texto</span>

In [ ]:
import re
import string

def limpiar_texto(texto):
    """
    Función para limpiar texto:
    - Convierte a minúsculas
    - Elimina puntuación
    - Elimina números
    - Elimina espacios extras
    """
    # Convertir a string por si acaso
    texto = str(texto)
    
    # Convertir a minúsculas
    texto = texto.lower()
    
    # Eliminar URLs
    texto = re.sub(r'http\S+|www\S+', '', texto)
    
    # Eliminar menciones y hashtags
    texto = re.sub(r'@\w+|#\w+', '', texto)
    
    # Eliminar números
    texto = re.sub(r'\d+', '', texto)
    
    # Eliminar puntuación
    texto = texto.translate(str.maketrans('', '', string.punctuation))
    
    # Eliminar espacios extras
    texto = ' '.join(texto.split())
    
    return texto

# Aplicar limpieza a ambos datasets
print("Limpiando textos...")
df_emotion_clean['sentence_clean'] = df_emotion_clean['sentence'].apply(limpiar_texto)
df_sentiment_clean['sentence_clean'] = df_sentiment_clean['sentence'].apply(limpiar_texto)

# Mostrar ejemplos
print("\nEjemplos de texto limpio (Emociones):")
for i in range(3):
    print(f"\nOriginal: {df_emotion_clean.iloc[i]['sentence']}")
    print(f"Limpio: {df_emotion_clean.iloc[i]['sentence_clean']}")

### <span style="color:#00bfff">4.3 Codificación de Variables Categóricas (Label Encoding)</span>

In [ ]:
# Label Encoding para emociones
label_encoder_emotion = LabelEncoder()
df_emotion_clean['emotion_encoded'] = label_encoder_emotion.fit_transform(df_emotion_clean['emotion'])

# Mostrar mapeo de emociones
print("Mapeo de Emociones:")
emotion_mapping = dict(zip(label_encoder_emotion.classes_, 
                           label_encoder_emotion.transform(label_encoder_emotion.classes_)))
for emotion, code in sorted(emotion_mapping.items(), key=lambda x: x[1]):
    print(f"{code}: {emotion}")

print("\n" + "="*60)

# Label Encoding para sentimientos
label_encoder_sentiment = LabelEncoder()
df_sentiment_clean['sentiment_encoded'] = label_encoder_sentiment.fit_transform(df_sentiment_clean['sentiment'])

# Mostrar mapeo de sentimientos
print("\nMapeo de Sentimientos:")
sentiment_mapping = dict(zip(label_encoder_sentiment.classes_, 
                             label_encoder_sentiment.transform(label_encoder_sentiment.classes_)))
for sentiment, code in sorted(sentiment_mapping.items(), key=lambda x: x[1]):
    print(f"{code}: {sentiment}")

## <span style="color:#00bfff">5. Vectorización de Texto (TF-IDF)</span>

Convertimos el texto en vectores numéricos usando TF-IDF (Term Frequency-Inverse Document Frequency)

In [ ]:
# Configurar TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,  # Limitar a las 5000 palabras más importantes
    min_df=2,           # Ignorar palabras que aparecen en menos de 2 documentos
    max_df=0.9,         # Ignorar palabras que aparecen en más del 90% de documentos
    ngram_range=(1, 2)  # Usar unigramas y bigramas
)

print("Vectorizando textos de emociones...")
X_emotion_tfidf = tfidf_vectorizer.fit_transform(df_emotion_clean['sentence_clean'])
print(f"Forma de la matriz TF-IDF (Emociones): {X_emotion_tfidf.shape}")
print(f"Número de características: {X_emotion_tfidf.shape[1]}")

print("\n" + "="*60)

# TF-IDF para sentimientos
tfidf_vectorizer_sentiment = TfidfVectorizer(
    max_features=5000,
    min_df=2,
    max_df=0.9,
    ngram_range=(1, 2)
)

print("\nVectorizando textos de sentimientos...")
X_sentiment_tfidf = tfidf_vectorizer_sentiment.fit_transform(df_sentiment_clean['sentence_clean'])
print(f"Forma de la matriz TF-IDF (Sentimientos): {X_sentiment_tfidf.shape}")
print(f"Número de características: {X_sentiment_tfidf.shape[1]}")

## <span style="color:#00bfff">6. División del Dataset en Entrenamiento y Prueba</span>

In [ ]:
# División para dataset de emociones (80% entrenamiento, 20% prueba)
X_train_emotion, X_test_emotion, y_train_emotion, y_test_emotion = train_test_split(
    X_emotion_tfidf, 
    df_emotion_clean['emotion_encoded'],
    test_size=0.2,
    random_state=42,
    stratify=df_emotion_clean['emotion_encoded']  # Mantener proporción de clases
)

print("Dataset de Emociones:")
print(f"Tamaño del conjunto de entrenamiento: {X_train_emotion.shape}")
print(f"Tamaño del conjunto de prueba: {X_test_emotion.shape}")

print("\n" + "="*60)

# División para dataset de sentimientos
X_train_sentiment, X_test_sentiment, y_train_sentiment, y_test_sentiment = train_test_split(
    X_sentiment_tfidf,
    df_sentiment_clean['sentiment_encoded'],
    test_size=0.2,
    random_state=42,
    stratify=df_sentiment_clean['sentiment_encoded']
)

print("\nDataset de Sentimientos:")
print(f"Tamaño del conjunto de entrenamiento: {X_train_sentiment.shape}")
print(f"Tamaño del conjunto de prueba: {X_test_sentiment.shape}")

## <span style="color:#00bfff">7. Entrenamiento de Modelos de Clasificación</span>

### <span style="color:#00bfff">7.1 Clasificación de Emociones</span>

Probaremos varios modelos de clasificación supervisada aprendidos en clase.

#### <span style="color:#00bfff">Modelo 1: k-Nearest Neighbors (k-NN)</span>

In [ ]:
# Configurar TF-IDF Vectorizer con variables de .env
tfidf_vectorizer = TfidfVectorizer(
    max_features=MAX_FEATURES,  # Desde .env
    min_df=MIN_DF,              # Desde .env
    max_df=MAX_DF,              # Desde .env
    ngram_range=NGRAM_RANGE     # Desde .env
)

print("Vectorizando textos de emociones...")
X_emotion_tfidf = tfidf_vectorizer.fit_transform(df_emotion_clean['sentence_clean'])
print(f"Forma de la matriz TF-IDF (Emociones): {X_emotion_tfidf.shape}")
print(f"Número de características: {X_emotion_tfidf.shape[1]}")

#### <span style="color:#00bfff">Modelo 2: Naive Bayes (Multinomial)</span>

In [ ]:
print("="*60)

# TF-IDF para sentimientos con variables de .env
tfidf_vectorizer_sentiment = TfidfVectorizer(
    max_features=MAX_FEATURES,
    min_df=MIN_DF,
    max_df=MAX_DF,
    ngram_range=NGRAM_RANGE
)

print("\nVectorizando textos de sentimientos...")
X_sentiment_tfidf = tfidf_vectorizer_sentiment.fit_transform(df_sentiment_clean['sentence_clean'])
print(f"Forma de la matriz TF-IDF (Sentimientos): {X_sentiment_tfidf.shape}")
print(f"Número de características: {X_sentiment_tfidf.shape[1]}")

#### <span style="color:#00bfff">Modelo 3: Regresión Logística</span>

In [ ]:
print("Entrenando modelo de Regresión Logística...")
lr_emotion = LogisticRegression(max_iter=1000, random_state=42)
lr_emotion.fit(X_train_emotion, y_train_emotion)

# Predicciones
y_pred_lr_emotion = lr_emotion.predict(X_test_emotion)

# Evaluación
accuracy_lr = accuracy_score(y_test_emotion, y_pred_lr_emotion)
print(f"\nAccuracy Regresión Logística (Emociones): {accuracy_lr:.4f}")
print("\nReporte de Clasificación:")
print(classification_report(y_test_emotion, y_pred_lr_emotion,
                          target_names=label_encoder_emotion.classes_))

#### <span style="color:#00bfff">Modelo 4: Random Forest</span>

In [ ]:
print("Entrenando modelo Random Forest...")
rf_emotion = RandomForestClassifier(n_estimators=100, random_state=42)
rf_emotion.fit(X_train_emotion, y_train_emotion)

# Predicciones
y_pred_rf_emotion = rf_emotion.predict(X_test_emotion)

# Evaluación
accuracy_rf = accuracy_score(y_test_emotion, y_pred_rf_emotion)
print(f"\nAccuracy Random Forest (Emociones): {accuracy_rf:.4f}")
print("\nReporte de Clasificación:")
print(classification_report(y_test_emotion, y_pred_rf_emotion,
                          target_names=label_encoder_emotion.classes_))

### <span style="color:#00bfff">7.2 Comparación de Modelos (Emociones)</span>

In [ ]:
# Crear DataFrame con resultados
resultados_emotion = pd.DataFrame({
    'Modelo': ['k-NN', 'Naive Bayes', 'Regresión Logística', 'Random Forest'],
    'Accuracy': [accuracy_knn, accuracy_nb, accuracy_lr, accuracy_rf]
})

resultados_emotion = resultados_emotion.sort_values('Accuracy', ascending=False)

print("\n" + "="*60)
print("COMPARACIÓN DE MODELOS - CLASIFICACIÓN DE EMOCIONES")
print("="*60)
display(resultados_emotion)

# Visualización
plt.figure(figsize=(10, 6))
plt.barh(resultados_emotion['Modelo'], resultados_emotion['Accuracy'], color='steelblue')
plt.xlabel('Accuracy', fontsize=12)
plt.title('Comparación de Modelos - Clasificación de Emociones', fontsize=14, fontweight='bold')
plt.xlim(0, 1)
for i, v in enumerate(resultados_emotion['Accuracy']):
    plt.text(v + 0.01, i, f"{v:.4f}", va='center')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

### <span style="color:#00bfff">7.3 Matriz de Confusión del Mejor Modelo (Emociones)</span>

In [ ]:
# División para dataset de emociones usando variables de .env
X_train_emotion, X_test_emotion, y_train_emotion, y_test_emotion = train_test_split(
    X_emotion_tfidf, 
    df_emotion_clean['emotion_encoded'],
    test_size=TEST_SIZE,           # Desde .env
    random_state=RANDOM_STATE,     # Desde .env
    stratify=df_emotion_clean['emotion_encoded']  # Mantener proporción de clases
)

print("Dataset de Emociones:")
print(f"Tamaño del conjunto de entrenamiento: {X_train_emotion.shape}")
print(f"Tamaño del conjunto de prueba: {X_test_emotion.shape}")

print("\n" + "="*60)

# División para dataset de sentimientos
X_train_sentiment, X_test_sentiment, y_train_sentiment, y_test_sentiment = train_test_split(
    X_sentiment_tfidf,
    df_sentiment_clean['sentiment_encoded'],
    test_size=TEST_SIZE,           # Desde .env
    random_state=RANDOM_STATE,     # Desde .env
    stratify=df_sentiment_clean['sentiment_encoded']
)

print("\nDataset de Sentimientos:")
print(f"Tamaño del conjunto de entrenamiento: {X_train_sentiment.shape}")
print(f"Tamaño del conjunto de prueba: {X_test_sentiment.shape}")

## <span style="color:#00bfff">8. Clasificación de Sentimientos</span>

Ahora aplicaremos los mismos modelos al dataset de sentimientos (positivo/negativo)

### <span style="color:#00bfff">8.1 Entrenamiento de Modelos</span>

In [ ]:
print("Entrenando modelo k-NN...")
knn_emotion = KNeighborsClassifier(n_neighbors=KNN_NEIGHBORS, metric='euclidean')  # Desde .env
knn_emotion.fit(X_train_emotion, y_train_emotion)

# Predicciones
y_pred_knn_emotion = knn_emotion.predict(X_test_emotion)

# Evaluación
accuracy_knn = accuracy_score(y_test_emotion, y_pred_knn_emotion)
print(f"\nAccuracy k-NN (Emociones): {accuracy_knn:.4f}")
print(f"Configuración: k={KNN_NEIGHBORS} vecinos")
print("\nReporte de Clasificación:")
print(classification_report(y_test_emotion, y_pred_knn_emotion, 
                          target_names=label_encoder_emotion.classes_))

### <span style="color:#00bfff">8.2 Comparación de Modelos (Sentimientos)</span>

In [ ]:
# Crear DataFrame con resultados
resultados_sentiment = pd.DataFrame({
    'Modelo': ['k-NN', 'Naive Bayes', 'Regresión Logística', 'Random Forest'],
    'Accuracy': [accuracy_knn_sent, accuracy_nb_sent, accuracy_lr_sent, accuracy_rf_sent]
})

resultados_sentiment = resultados_sentiment.sort_values('Accuracy', ascending=False)

print("\n" + "="*60)
print("COMPARACIÓN DE MODELOS - CLASIFICACIÓN DE SENTIMIENTOS")
print("="*60)
display(resultados_sentiment)

# Visualización
plt.figure(figsize=(10, 6))
plt.barh(resultados_sentiment['Modelo'], resultados_sentiment['Accuracy'], color='coral')
plt.xlabel('Accuracy', fontsize=12)
plt.title('Comparación de Modelos - Clasificación de Sentimientos', fontsize=14, fontweight='bold')
plt.xlim(0, 1)
for i, v in enumerate(resultados_sentiment['Accuracy']):
    plt.text(v + 0.01, i, f"{v:.4f}", va='center')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

### <span style="color:#00bfff">8.3 Reporte Detallado del Mejor Modelo (Sentimientos)</span>

In [ ]:
print("Entrenando modelo de Regresión Logística...")
lr_emotion = LogisticRegression(max_iter=LR_MAX_ITER, random_state=RANDOM_STATE)  # Desde .env
lr_emotion.fit(X_train_emotion, y_train_emotion)

# Predicciones
y_pred_lr_emotion = lr_emotion.predict(X_test_emotion)

# Evaluación
accuracy_lr = accuracy_score(y_test_emotion, y_pred_lr_emotion)
print(f"\nAccuracy Regresión Logística (Emociones): {accuracy_lr:.4f}")
print(f"Configuración: max_iter={LR_MAX_ITER}, random_state={RANDOM_STATE}")
print("\nReporte de Clasificación:")
print(classification_report(y_test_emotion, y_pred_lr_emotion,
                          target_names=label_encoder_emotion.classes_))

## <span style="color:#00bfff">9. Predicciones con Nuevos Textos</span>

Probemos los modelos entrenados con nuevos ejemplos

In [ ]:
print("Entrenando modelo Random Forest...")
rf_emotion = RandomForestClassifier(n_estimators=RF_ESTIMATORS, random_state=RANDOM_STATE)  # Desde .env
rf_emotion.fit(X_train_emotion, y_train_emotion)

# Predicciones
y_pred_rf_emotion = rf_emotion.predict(X_test_emotion)

# Evaluación
accuracy_rf = accuracy_score(y_test_emotion, y_pred_rf_emotion)
print(f"\nAccuracy Random Forest (Emociones): {accuracy_rf:.4f}")
print(f"Configuración: n_estimators={RF_ESTIMATORS}, random_state={RANDOM_STATE}")
print("\nReporte de Clasificación:")
print(classification_report(y_test_emotion, y_pred_rf_emotion,
                          target_names=label_encoder_emotion.classes_))

## <span style="color:#00bfff">10. Conclusiones</span>

In [ ]:
print("="*80)
print("RESUMEN DEL PROYECTO")
print("="*80)

print("\n📊 DATASET DE EMOCIONES:")
print(f"   - Total de muestras: {len(df_emotion_clean)}")
print(f"   - Número de emociones: {len(df_emotion_clean['emotion'].unique())}")
print(f"   - Emociones: {', '.join(sorted(df_emotion_clean['emotion'].unique()))}")
print(f"   - Mejor modelo: {mejor_modelo_emotion}")
print(f"   - Mejor accuracy: {resultados_emotion.iloc[0]['Accuracy']:.4f}")

print("\n📊 DATASET DE SENTIMIENTOS:")
print(f"   - Total de muestras: {len(df_sentiment_clean)}")
print(f"   - Clases: {', '.join(sorted(df_sentiment_clean['sentiment'].unique()))}")
print(f"   - Mejor modelo: {mejor_modelo_sentiment}")
print(f"   - Mejor accuracy: {resultados_sentiment.iloc[0]['Accuracy']:.4f}")

print("\n✅ TÉCNICAS APLICADAS DEL CURSO:")
print("   1. ✓ Análisis Exploratorio de Datos (EDA)")
print("   2. ✓ Limpieza de datos (valores nulos)")
print("   3. ✓ Preprocesamiento de texto")
print("   4. ✓ Label Encoding para variables categóricas")
print("   5. ✓ División de datos (train/test split)")
print("   6. ✓ Vectorización de texto (TF-IDF)")
print("   7. ✓ Clasificación supervisada con múltiples modelos")
print("   8. ✓ Evaluación con métricas (accuracy, precision, recall, f1-score)")
print("   9. ✓ Matrices de confusión")
print("   10. ✓ Comparación y selección del mejor modelo")

print("\n" + "="*80)

## <span style="color:#00bfff">11. Exportar Modelos (Opcional)</span>

In [ ]:
import pickle

# Guardar el mejor modelo de emociones
with open('mejor_modelo_emociones.pkl', 'wb') as f:
    pickle.dump(modelo_prediccion, f)

# Guardar vectorizador
with open('tfidf_vectorizer_emociones.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

# Guardar label encoder
with open('label_encoder_emociones.pkl', 'wb') as f:
    pickle.dump(label_encoder_emotion, f)

print("✅ Modelos guardados exitosamente!")
print("   - mejor_modelo_emociones.pkl")
print("   - tfidf_vectorizer_emociones.pkl")
print("   - label_encoder_emociones.pkl")